<a href="https://colab.research.google.com/github/Dwslr/Simulative_mod6_basic_python_practice_cases/blob/main/Copy_of_%D0%9A%D0%B5%D0%B9%D1%81_%D0%9B%D0%BE%D0%B3%D0%B8_%D0%93%D0%BB%D0%B0%D0%B2%D0%B0_14_simulative_module6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1 кейс**

**Работа с логами**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [ ]:
!wget https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log

### **Решения**

#### **Задача 1**

Ваша задача написать функцию `count_success_and_failure`, которая принимает на вход путь к файлу с логами и подсчитывает количество успешных продлений и ошибок при списании. Функция должна вернуть кортеж из двух значений: количества успешных попыток и неуспешных.

**Решение**

Напишите свое решение ниже

In [3]:
# v1

# v2
def count_success_and_failure2(file_path):
  with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()

    all_atts = 0
    err_atts = 0

    for line in lines:
      line = [element.strip() for element in line.split('|')]
      all_atts += 1 if line[4].startswith('[demon] Обновляем') else 0
      err_atts += 1 if line[0].lower().startswith('err') else 0

  return all_atts - err_atts, err_atts


#v3
def count_success_and_failure(logs_path):
  with open(logs_path, 'r') as logs_read:
    lines = logs_read.readlines()

    lines = [line.split(' | ') for line in lines]

    #[print(line[3:5]) for line in lines if line[0].strip() == 'ERROR']  #line[4].strip().startswith('[demon] Обновляем')]

    attempts = sum([1 for line in lines if line[4].strip().startswith('[demon] Обновляем')])
    errors = sum([1 for line in lines if line[0].lower().strip() == 'error'])
    success = attempts - errors

    return success, errors


#v4
def count_success_and_failure4(logs_path):
  with open(logs_path, 'r') as logs_read:
    lines = logs_read.readlines()

    lines = [line.split(' | ') for line in lines]

    # filtered = list(filter(lambda x: x[4].lower().strip().startswith('[demon] обновляем'), lines))
    # print(filtered)

    atts = sum(map(lambda x: 1 if x[4].strip().startswith('[demon] Обновляем') else 0, lines))
    errs = sum(map(lambda x: 1 if x[0].strip() == 'ERROR' else 0, lines))

    return atts - errs, errs


count_success_and_failure('auto_purchase.log')

(1034, 186)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [4]:
res = count_success_and_failure('auto_purchase.log')

try:
    assert res == (1034, 186)
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Ваша задача написать функцию `auto_renewal_sub`, которая принимает на вход путь к файлу с логами и обрабатывает количество клиентов с автопродлением подписки. Мы хотим посмотреть на изменение этого показателя в динамике: посчитайте сглаженные значения с помощью метода скользящего среднего и метода медианного сглаживания.  

**Примечание:** При сглаживании берем все предыдущие значения, включая текущее, будущие значения не берем.

Функция должна записать в файл `auto_renewal_sub.txt` два списка, предварив их соответствущими обозначениями:

`Среднее: [2.0, 1.0, 0.67...]`

`Медиана: [2, 2, 0...]`

**Решение**

Напишите свое решение ниже

In [9]:
#v2 new

def auto_renewal_sub(log_path):
  with open(log_path, 'r') as read_logs:
    lines = read_logs.readlines()

    lines = [[el.strip() for el in line.split(' | ')] for line in lines]
    lines = [line[:1] + [line[1].split()[0]] + line[4:] for line in lines]

    # for line in lines: #check
    #   print(line)

    daysub_d = {}

    daysub_d.update(
        (line[1], daysub_d.get(line[1], 0) + 1) if line[2].startswith('[demon] Обновляем')
        else (line[1], daysub_d.get(line[1], 0) - 1) if line[0] == 'ERROR'
        else (line[1], daysub_d.get(line[1], 0)) for line in lines
        )
    #print(daysub_d) #check

    lst = list(daysub_d.values())
    #print(lst) #check

    res = []

    avg = []
    med = []

    for n in lst:
      res.append(n)
      res.sort()

      avg.append(round(sum(res) / len(res), 2))

      med.append(
          res[len(res) // 2] if len(res) % 2 != 0
          else round((res[len(res) // 2 - 1] + res[len(res) // 2]) / 2)
      )

    #print(avg)
    #print(med)

    with open('auto_renewal_sub.txt', 'w') as write_txt:
      write_txt.write(f'Среднее: {avg}\nМедиана: {med}')


auto_renewal_sub('auto_purchase.log')

In [6]:
#v1

import re
from datetime import datetime
#from statistics import mean

def auto_renewal_sub(log_file_path):
  with open(log_file_path, 'r', newline='') as lr:
    data = lr.readlines()

    pays_d = {}

    for line in data:
      line = line.strip().split(' | ')
      if line[0].lower().startswith('inf'):
        date = line[1].split()[0]
        if date not in pays_d:
            pays_d[date] = 1 if line[4].lower().startswith('[demon] обновляем подписку') else 0
        else:
            pays_d[date] += 1 if line[4].lower().startswith('[demon] обновляем подписку') else 0

    #print(pays_d)
    #print(pays_d.keys())

    pays = []

    for date, regs in pays_d.items():
      pays.append(regs)

    res = []

    summ = 0
    avg = []
    mdn = []

    for pay in pays:
      res.append(pay)
      res.sort()

      avg.append(round(sum(res) / len(res), 2))
      #avg.append(mean(res))

      if len(res) % 2 != 0:
        mdn.append(res[len(res) // 2])
      else:
        mdn.append(round((res[len(res) // 2 - 1] + res[len(res) // 2]) / 2))

    #return f'{avg}\n{mdn}'
    with open('auto_renewal_sub.txt', 'w') as wr:
      wr.write(f'Среднее: {avg}\nМедиана: {mdn}')

res = auto_renewal_sub('auto_purchase.log')

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt

import pandas as pd

user_answer = pd.read_csv('auto_renewal_sub.txt')
correct_answer = pd.read_csv('cor_auto_renewal.txt')

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

AssertionError: При проверке возникла ошибка ValueError('Can only compare identically-labeled DataFrame objects')

#### **Задача 3**

Напишите функцию `sub_renewal_by_day`, которая принимает на вход путь к файлу с логами и анализирует взаимосвязь дня продления подписки и количества продлений в этот день. Функция должна записать в файл `weekdays.txt` аналитическую записку в формате:

**`Количество обновлений подписки по дням недели:`**

**`Понедельник: 6`**

**`Вторник: 7`**

**`Среда: 8`**

**`...`**

**Решение**

Напишите свое решение ниже

In [10]:
from datetime import datetime


def sub_renewal_by_day(file_path):
  with open(file_path, 'r') as rf:
    data = rf.readlines()

    weeknames_d = {'Monday': 0,
                   'Tuesday': 0,
                   'Wednesday': 0,
                   'Thursday': 0,
                   'Friday': 0,
                   'Saturday': 0,
                   'Sunday': 0}

    for row in data:
        row = [el.strip() for el in row.split(' | ')]

        if row[0] == 'ERROR' or row[4].startswith('[demon] Обновляем'):
          date = datetime.strptime(row[1], '%Y-%m-%d %H:%M:%S,%f') # convert string into date
          dayname = date.strftime('%A') # convert date into weekday name

          if dayname in weeknames_d and row[0] != 'ERROR':
            weeknames_d[dayname] += 1
          else:
            weeknames_d[dayname] -= 1

    print(weeknames_d)


    with open('weekdays.txt', 'w') as wf:
      wf.write('Количество обновлений подписки по дням недели:\n')
      for day, cnt in weeknames_d.items():
        wf.write(f'{day}: {cnt}\n')


sub_renewal_by_day('auto_purchase.log')

{'Monday': 136, 'Tuesday': 144, 'Wednesday': 162, 'Thursday': 169, 'Friday': 145, 'Saturday': 135, 'Sunday': 143}


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt

import pandas as pd

user_answer = pd.read_csv('weekdays.txt')
correct_answer = pd.read_csv('cor_weekdays.txt')

In [12]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

AssertionError: При проверке возникла ошибка AssertionError('Ответы не совпадают')